In [1]:
#This book obtains the star counts of every star detected in each image and saves to a csv file
import pandas as pd
import numpy as np
import inspect
import warnings
import matplotlib.pyplot as plt
import os
import glob
import time

from astropy.nddata import extract_array
from astropy.table import QTable
from astropy.utils import lazyproperty

from photutils.detection.core import StarFinderBase, _StarFinderKernel
from photutils.utils._convolution import _filter_data
from photutils.utils._misc import _get_version_info
from photutils.utils.exceptions import NoDetectionsWarning
from photutils.aperture import aperture_photometry, CircularAperture
from photutils.aperture import CircularAnnulus
from photutils.aperture import CircularAperture, ApertureStats
from photutils.datasets import make_4gaussians_image

import time
from matplotlib.colors import LogNorm
from IPython import display
from IPython.display import clear_output

In [3]:
#Getting all the csv files for each star image:
image = []
image_list = glob.glob('Reduced_star_Light_Blue*.fits')
image_list

['Reduced_star_Light_Blue_30_secs_001.fits',
 'Reduced_star_Light_Blue_30_secs_002.fits',
 'Reduced_star_Light_Blue_30_secs_003.fits',
 'Reduced_star_Light_Blue_30_secs_004.fits',
 'Reduced_star_Light_Blue_30_secs_005.fits',
 'Reduced_star_Light_Blue_30_secs_006.fits',
 'Reduced_star_Light_Blue_30_secs_007.fits',
 'Reduced_star_Light_Blue_30_secs_008.fits',
 'Reduced_star_Light_Blue_30_secs_009.fits',
 'Reduced_star_Light_Blue_30_secs_010.fits',
 'Reduced_star_Light_Blue_30_secs_011.fits',
 'Reduced_star_Light_Blue_30_secs_012.fits',
 'Reduced_star_Light_Blue_30_secs_013.fits',
 'Reduced_star_Light_Blue_30_secs_014.fits',
 'Reduced_star_Light_Blue_30_secs_015.fits',
 'Reduced_star_Light_Blue_30_secs_016.fits',
 'Reduced_star_Light_Blue_30_secs_017.fits',
 'Reduced_star_Light_Blue_30_secs_018.fits',
 'Reduced_star_Light_Blue_30_secs_019.fits',
 'Reduced_star_Light_Blue_30_secs_020.fits',
 'Reduced_star_Light_Blue_30_secs_021.fits',
 'Reduced_star_Light_Blue_30_secs_022.fits',
 'Reduced_

In [4]:
#Loading up the first reduced file:
from astropy.io import fits
for image in image_list:
    hdul = fits.open(image)
    hdul.info()

Filename: Reduced_star_Light_Blue_30_secs_001.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_002.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_003.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_004.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_005.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_006.fits
No.    Name      Ver    Type      Cards   Di

Filename: Reduced_star_Light_Blue_30_secs_047.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_048.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_049.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_050.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_051.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      42   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_30_secs_052.fits
No.    Name      Ver    Type      Cards   Di

Filename: Reduced_star_Light_Blue_45_secs_016.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_45_secs_017.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_45_secs_018.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_45_secs_019.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_45_secs_020.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      55   (4656, 3520)   float64   
Filename: Reduced_star_Light_Blue_45_secs_021.fits
No.    Name      Ver    Type      Cards   Di

In [5]:
for image in image_list:
    hdul1 = fits.getdata(image)
    print(hdul1) #Shows the array of the image

[[0.30879497 0.49253911 0.8116568  ... 0.49097476 0.50397678 0.47830725]
 [0.30879497 0.49253911 0.76735036 ... 0.82715059 0.68400098 0.47290618]
 [0.30744321 0.5555392  0.63574453 ... 0.43814211 0.60587021 0.07136463]
 ...
 [0.3516012  0.50039541 0.4285577  ... 0.34503314 0.41739285 0.91079941]
 [0.62162319 0.48642715 0.51558957 ... 0.45584788 0.71317517 0.60092364]
 [0.7874809  0.48693675 0.59892247 ... 0.42815942 0.71317517 0.60092364]]
[[0.6687891  0.5298904  0.73351269 ... 0.60762662 0.5768058  0.7895701 ]
 [0.6687891  0.5298904  0.65029692 ... 0.60265254 0.52854621 0.78817696]
 [0.60738781 0.67796459 0.25091128 ... 0.67092128 0.26364227 0.56484349]
 ...
 [0.54352325 0.2004581  0.50508586 ... 0.49375432 0.66668891 0.60309691]
 [0.43117491 0.70820244 0.69902989 ... 0.53131938 0.29529909 0.41059732]
 [0.44955998 0.29350319 0.71341488 ... 0.65377036 0.29529909 0.41059732]]
[[0.26879562 0.67929559 0.69444064 ... 0.49097476 0.39473326 0.47830725]
 [0.26879562 0.67929559 0.96243943 ... 

[[0.22879627 0.45518781 0.10835983 ... 0.37432289 0.17624622 0.43939939]
 [0.22879627 0.45518781 0.14306532 ... 0.15365644 0.21763667 0.27586194]
 [0.15747091 0.59634767 0.02001133 ... 0.282956   0.11154096 0.10932455]
 ...
 [0.12129474 0.01299728 0.27550138 ... 0.30785284 0.3461654  0.25693159]
 [0.3549956  0.3755395  0.25877314 ... 0.15396187 0.33328783 0.29640153]
 [0.03654552 0.25481648 0.25544527 ... 0.20254849 0.33328783 0.29640153]]
[[0.18879692 0.30578262 0.18650394 ... 0.29655497 0.35831875 0.20595225]
 [0.18879692 0.30578262 0.3381544  ... 0.41557083 0.21763667 0.11822654]
 [0.15747091 0.35149689 0.21242796 ... 0.39934559 0.26364227 0.03340472]
 ...
 [0.27483238 0.23795026 0.2372373  ... 0.08477107 0.2037105  0.21846878]
 [0.27881628 0.26465185 0.2954612  ... 0.22943337 0.33328783 0.25833626]
 [0.2993729  0.10006963 0.25544527 ... 0.08974302 0.33328783 0.25833626]]
[[0.14879757 0.26843132 0.3037201  ... 0.41320684 0.46756228 0.32267582]
 [0.14879757 0.26843132 0.18208314 ... 

[[0.18879692 0.30578262 0.18650394 ... 0.14101915 0.1034172  0.08922868]
 [0.18879692 0.30578262 0.41619003 ... 0.41557083 0.21763667 0.31527078]
 [0.30744321 0.39230535 0.25091128 ... 0.16656642 0.26364227 0.14728446]
 ...
 [0.23644797 0.2004581  0.27550138 ... 0.23349225 0.27493795 0.10308034]
 [0.31690594 0.19072675 0.14870895 ... 0.30490487 0.29529909 0.37253205]
 [0.26182614 0.29350319 0.36993767 ... 0.27775213 0.29529909 0.37253205]]
[[ 0.38879366  0.45518781  0.53815242 ...  0.49097476  0.06700269
   0.24486011]
 [ 0.38879366  0.45518781 -0.01300594 ...  0.45298717  0.25650037
   0.19704424]
 [ 0.15747091  0.2290715   0.05849465 ...  0.16656642  0.11154096
   0.26116419]
 ...
 [ 0.31321679  0.16296594  0.31376546 ...  0.30785284  0.2037105
   0.21846878]
 [ 0.27881628  0.33857695  0.25877314 ...  0.19169762  0.29529909
   0.220271  ]
 [ 0.1867326   0.29350319  0.179117   ...  0.24015031  0.29529909
   0.220271  ]]
[[0.06879888 0.26843132 0.22557599 ... 0.10213519 0.32190425 0.36

[[0.31999478 0.4980173  0.49491268 ... 0.36291693 0.50980311 0.38907857]
 [0.31999478 0.4980173  0.28613064 ... 0.4489961  0.62181907 0.44663361]
 [0.39992613 0.40808463 0.5644221  ... 0.59488008 0.43095371 0.27837269]
 ...
 [0.46061293 0.54988506 0.33162203 ... 0.4709504  0.30865228 0.58976313]
 [0.20314483 0.36962549 0.2935045  ... 0.47798618 0.32923569 0.32989896]
 [0.40049887 0.41265826 0.50885512 ... 0.37601823 0.32923569 0.32989896]]
[[0.45332595 0.4980173  0.33862447 ... 0.54437539 0.26703972 0.44095571]
 [0.45332595 0.4980173  0.52023753 ... 0.34921919 0.69954645 0.34154335]
 [0.24995383 0.48970155 0.33352215 ... 0.41382962 0.5577048  0.50613216]
 ...
 [0.40943371 0.39991641 0.48467835 ... 0.19829491 0.52233462 0.33334438]
 [0.53325519 0.34498379 0.41579804 ... 0.47798618 0.35456152 0.45678317]
 [0.50062359 0.72215196 0.45796961 ... 0.17547517 0.35456152 0.45678317]]
[[0.50665841 0.29881038 0.6512009  ... 0.23330374 0.38842141 0.33720142]
 [0.50665841 0.29881038 0.13005938 ... 

[[0.47999218 0.62252162 0.18233625 ... 0.49253011 0.46125043 0.64846428]
 [0.47999218 0.62252162 0.39017815 ... 0.42405187 0.4663643  0.604269  ]
 [0.32493998 0.38087898 0.64138875 ... 0.67247314 0.50700436 0.63266519]
 ...
 [0.66532978 0.24994776 0.30611264 ... 0.52052413 0.23742483 0.51283751]
 [0.40628967 0.41890889 0.36688063 ... 0.50314335 0.58249392 0.45678317]
 [0.45056123 0.49003169 0.33075583 ... 0.37601823 0.58249392 0.45678317]]
[[0.58665711 0.62252162 0.28652839 ... 0.75175649 0.60690846 0.59658713]
 [0.58665711 0.62252162 0.98845131 ... 0.77327106 0.44045517 0.84072209]
 [0.4499169  0.43529027 0.69269985 ... 0.28450787 0.70980611 0.55674537]
 ...
 [0.48620253 0.44990596 0.48467835 ... 0.57009786 0.49859214 0.79489814]
 [0.55864829 0.51747569 0.70930254 ... 0.65408635 0.25325823 0.38065264]
 [0.45056123 0.56740511 0.6360689  ... 0.40108611 0.25325823 0.38065264]]
[[0.45332595 0.4980173  0.52096072 ... 0.38883956 0.48552677 0.46689428]
 [0.45332595 0.4980173  0.67630879 ... 

In [6]:
#Finding the date and time of observation:
from astropy.time import Time
for image in image_list:
    hdul2 = fits.open(image)
    date = hdul2[0].header.get('DATE-OBS')
    t = Time(date)
    t.format = 'jd'
    MJD = t - 2400000.5
    print(MJD)

59967.87310986111
59967.874653425926
59967.87618432871
59967.87773270833
59967.879253541665
59967.88078103009
59967.882345590275
59967.88387153935
59967.885386967595
59967.88690265046
59967.888447592595
59967.889992083336
59967.89152123842
59967.89304652778
59967.89456148148
59967.89608049769
59967.897613113426
59967.89915074074
59967.90068165509
59967.90220476852
59967.903709907405
59967.905243518515
59967.90676453704
59967.90828326389
59967.90981111111
59967.91132849537
59967.91287148148
59967.91439060185
59967.915910578704


C:\Users\micha\anaconda3\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


59967.91743504629
59967.91897384259
59967.92054478009
59967.93146984954
59967.933007280095
59967.93455017361
59967.93609423611
59967.937624224534
59967.94140865741
59967.94293320602
59967.94445840278
59967.94601021991
59967.94753548611
59967.94906466435
59967.95059148148
59967.95212457176
59967.9536387037
59967.95516107639
59967.95671788194
59967.95828792824
59967.95981550926
59967.962827766205
59967.96442243055
59967.965937800924
59967.96747443287
59967.968985
59967.97050858796
59967.972044699076
59967.97358300926
59967.97510704861
59967.97661938657
59967.97815052083
59967.97970297454
59967.98123207176
59967.982759479164
59967.98427847222
59967.985823958334
59967.98734803241
59967.98887278935
59967.990398587965
59967.99192008102
59967.993447719906
59967.9949853588
59967.99652748842
59967.99805303241
59967.999578414354
59968.00110577546
59968.00263487268
59967.78169043981
59967.78384596065
59967.78601425926
59967.78815648148
59967.79015173611
59967.7921571412
59967.794350358796
59967.7

In [7]:
for image in image_list:
    hdul1 = fits.getdata(image)

In [8]:
from photutils.detection import DAOStarFinder
for image in image_list:
    hdul1 = fits.getdata(image)
    daofind = DAOStarFinder(fwhm=6,threshold = 5*np.median(hdul1),brightest=100)
    df_out = daofind(hdul1 - np.median(hdul1))
    
    #Converting the table of data to a dataframe:
    stars_found = df_out.to_pandas()
    
    #Circular apertures will be drawn around all of the stars:
    positions = np.transpose((stars_found['xcentroid'], stars_found['ycentroid']))  
    apertures = CircularAperture(positions, r = 21)  
    phot_table = aperture_photometry(hdul1, apertures)  
    for col in phot_table.colnames:  
        phot_table[col].info.format = '%.8g'  # for consistent table output
    #print(phot_table)
    
    #Showing in the image:
    import matplotlib.pyplot as plt
    #plt.figure()
    #plt.imshow(hdul1, cmap='gray_r', origin='lower')
    #apertures.plot(color='blue', lw=1.5, alpha=0.5)
    
    #Finding the stats for the apertures:
    CIRC_stats = ApertureStats(hdul1, apertures)
    df_star_stats = pd.DataFrame()
    df_star_stats['Area of circular aperture'] = CIRC_stats.sum_aper_area.value
    df_star_stats['Sum of circular aperture'] = CIRC_stats.sum
    #df_star_stats
    
    #Circular annuli will be drawn around all of the stars:
    positions = np.transpose((stars_found['xcentroid'], stars_found['ycentroid']))  
    annuli = CircularAnnulus(positions, r_in = 21, r_out = 21*np.sqrt(5))  
    phot_table_2 = aperture_photometry(hdul1, annuli)  
    for col in phot_table_2.colnames:  
        phot_table_2[col].info.format = '%.8g'  # for consistent table output
    #print(phot_table_2)
    
    #Showing in the image:
    import matplotlib.pyplot as plt
    #plt.imshow(hdul1, cmap='gray_r', origin='lower')
    #annuli.plot(color='green', lw=1.5, alpha=0.5)
    
    #Finding the required annulus stats:
    annulus_stats = ApertureStats(hdul1, annuli)
    df_star_stats['Median of annulus'] = annulus_stats.median
    df_star_stats['id'] = annulus_stats.id
    #df_star_stats
    
    #Calculating the star counts:
    def star_counts():
        Ct = df_star_stats['Sum of circular aperture']
        Ca = df_star_stats['Area of circular aperture']
        P = df_star_stats['Median of annulus']
        star_counts = Ct - (P*Ca)
        return star_counts
    df_star_stats['star counts'] = star_counts ()
    #df_star_stats
    
    #Collect all necessary data into one dataframe and save to a .csv file:
    hdul2 = fits.open(image)
    date = hdul2[0].header.get('DATE-OBS')
    t = Time(date)
    t.format = 'jd'
    MJD = t.jd - 2400000.5
    #print(MJD)
    
    exp_time = hdul2[0].header.get('EXPTIME')
    df_star_stats['exp_time'] = exp_time
    df_star_stats['MJD'] = MJD
    df_star_stats['xcentroid'] = stars_found['xcentroid'] 
    df_star_stats['ycentroid'] = stars_found['ycentroid']
    df_star_stats    
    df_star_stats.to_csv(image+'star_stats.csv')